In [7]:
import basic_image_operations as basic_ops
import filters
import geometrical_image_operations as geo_ops
import colors
import numpy as np
import color_detection as cd

'''stop_img = basic_ops.load_image(image_path='traffic_sign_templates/stop_signs/stop.jpg')
vf_img = basic_ops.load_image(image_path='traffic_sign_templates/vf_signs/vf.jpg')
vfa_img = basic_ops.load_image(image_path='traffic_sign_templates/vfa_signs/vfa.jpg')
vfs_img = basic_ops.load_image(image_path='traffic_sign_templates/vfs_signs/vfs.jpg')'''
#images = basic_ops.load_images(folder_path='data', amount=3)
images = basic_ops.load_images(folder_path='traffic_sign_images/vfs')
images = [geo_ops.resize_image(image=image, target_height=image.shape[0]//8, target_width=image.shape[1]//8) for image in images]

Image loaded from traffic_sign_images/vfs/vfs_06.jpg
Image loaded from traffic_sign_images/vfs/vfs_03.jpg
Image loaded from traffic_sign_images/vfs/vfs_01.jpg
Image loaded from traffic_sign_images/vfs/vfs_07.jpg
Image loaded from traffic_sign_images/vfs/vfs_02.jpg
Image loaded from traffic_sign_images/vfs/vfs_05.jpg
Image loaded from traffic_sign_images/vfs/vfs_04.jpg


In [8]:
for i, image in enumerate(images):
    image = filters.median_filter(image=image, dim=3)
    #image = stat_ops.gamma_equalization(image=image, gamma=0.25)
    images[i] = image

In [9]:
import importlib
import colors
import bounding_box
importlib.reload(colors)
importlib.reload(bounding_box)
color_functions = [colors.is_strong_red, colors.is_strong_yellow, colors.is_strong_blue]

color_bounding_boxes = []
color_images = images.copy()
for i, image in enumerate(color_images):
    image_bounding_boxes = []
    img = np.copy(image)
    for color_function in color_functions:
        mask = colors.get_mask(image=img, color_function=color_function)
        basic_ops.show_image(filters.black_white_filter(mask, threshold=1))
        blobs = cd.get_blobs(mask=mask)
        img, bounding_boxes = bounding_box.draw_bounding_boxes(image=img, blobs=blobs, min_box_area=200, box_color=colors.get_color_from_function(color_function))
        image_bounding_boxes.extend(bounding_boxes)
    color_bounding_boxes.extend(image_bounding_boxes)
    color_images[i] = img

Image displayed
Image displayed
Image displayed
Image displayed
Image displayed
Image displayed
Image displayed
Image displayed
Image displayed
Image displayed
Image displayed
Image displayed
Image displayed
Image displayed
Image displayed
Image displayed
Image displayed
Image displayed
Image displayed
Image displayed
Image displayed


In [10]:
for i, image in enumerate(color_images):
    p2, p98 = np.percentile(image, (2, 98))
    image = np.clip((image - p2) * 255.0 / (p98 - p2), 0, 255).astype(np.uint8)
    basic_ops.show_image(image=image)
    color_images[i] = image

Image displayed
Image displayed
Image displayed
Image displayed
Image displayed
Image displayed
Image displayed
